# Programação orientada a objetos

Este notebook é a quarta parte da série sobre Programação orientada a objetos e é baseado nos slides ["Design patterns in Python"](http://www.aleax.it/gdd_pydp.pdf), de Alex Martelli.

## O que vem antes de padrões de projeto

Apesar de ter se tornado o paradigma de programação dominante das últimas décadas, algumas das promessas de POO não se traduzem em benefícios práticos:

> The problem with object-oriented languages is they’ve got all this implicit environment that they carry around with them. You wanted a banana but what you got was a gorilla holding the banana and the entire jungle.

Joe Armstrong, criador de Erlang. Tradução livre:

> O problema com linguagens orientadas a objetos é que elas carregam todo esse ambiente implícito com elas. Você queria uma banana mas o que você recebeu foi um gorila segurando uma banana e a selva inteira.

Neste notebook, vamos começar com uma discussão sobre **composição** como uma alternativa a herança e em seguida vamos discutir como Python lida com polimorfismo de forma mais simples que outras linguagens, fazendo uso de **protocolos**.

## Herança x composição

Uma das discussões mais centrais em relação às vantagens de POO gira em torno de herança. Em sistemas grandes, uma classe facilmente se perde no meio de uma hierarquia significativa, levando ao problema da origem Jedi:

> Rey tem poderes Jedi, então algum Jedi deve fazer parte de sua linhagem.

Em grandes frameworks, encontrar quem é o ancestral Jedi de Rey é um desafio significativo. Uma das formas de amenizar esse problema é entendendo a diferença de relação entre classes.

### Relação é-um

Uma relação do tipo **é-um** (em inglês, **is-a**) é geralmente representada através de herança. Vamos tomar como exemplo uma classe pra representar uma pessoa.

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return "Name: {}".format(self.name)
    
    def hello(self):
        print("Hi, I'm {}".format(self.name))

Essa classe é bastante geral, o que indica que muitas classes especializadas podem reusar seu código. Vamos ver o caso da classe que representa um contribuinte:

In [2]:
class TaxPayer(Person):
    def __init__(self, name, cpf, income):
        super().__init__(name)
        self.cpf = cpf
        self.income = float(income)
        
    def __str__(self):
        return "{}, CPF: {}, Renda: {}, Imposto a pagar: {}".format(super().__str__(), self.cpf, self.income, self.duetax)
    
    @property
    def duetax(self):
        return self.income * 0.2

In [3]:
clovis = TaxPayer("Clóvis", "000.111.222-33", 1000)

In [4]:
print(clovis)

Name: Clóvis, CPF: 000.111.222-33, Renda: 1000.0, Imposto a pagar: 200.0


Dependendo da aplicação, pode ser mais útil modelar uma classe contato, para ser usado em agendas:

In [5]:
class Contact(Person):
    def __init__(self, name, mail, phone):
        super().__init__(name)
        self.mail = mail
        self.phone = phone
    
    def __str__(self):
        return "{}, Mail: {}, Telefone: {}".format(super().__str__(), self.mail, self.phone)

In [6]:
nilson = Contact("Nilson", "ni@ls.on", "001122334455")

In [7]:
print(nilson)

Name: Nilson, Mail: ni@ls.on, Telefone: 001122334455


### Relação tem-um

Uma relação do tipo **tem-um** (em inglês, **has-a**) é geralmente representada através de composição, onde uma classe contém um objeto de outra classe. 

Vamos revisar o exemplo sob esta perspectiva. Neste caso, em vez de dizer que um contribuinte ou um contato são pessoas, vamos dizer que pessoas têm um registro de contribuinte e um cartão de contatos:

In [8]:
class TaxRecord:
    def __init__(self, cpf, income):
        self.cpf = cpf
        self.income = float(income)
        
    def __str__(self):
        return "CPF: {}, Renda: {}, Imposto a pagar: {}".format(self.cpf, self.income, self.duetax)
    
    @property
    def duetax(self):
        return self.income * 0.2

In [9]:
class ContactCard:
    def __init__(self, mail, phone):
        self.mail = mail
        self.phone = phone
    
    def __str__(self):
        return "Mail: {}, Telefone: {}".format(self.mail, self.phone)

In [10]:
class Person:
    def __init__(self, name, tax_record=None, contact_card=None):
        self.name = name
        self.tax_record = tax_record
        self.contact_card = contact_card
    
    def __str__(self):
        base_string = "Name: {}".format(self.name)
        if self.tax_record:
            base_string += ", {}".format(str(self.tax_record))
        if self.contact_card:
            base_string += ", {}".format(str(self.contact_card))
        return base_string
    
    def hello(self):
        print("Hi, I'm {}".format(self.name))

In [11]:
card = ContactCard("th@an.os", "001122334455")
thanos = Person("Thanos", contact_card= card)

In [12]:
print(thanos)

Name: Thanos, Mail: th@an.os, Telefone: 001122334455


**O exemplo acima foca apenas na relação entre as classes. Em uma situação real, o uso de composição levaria ao uso de builders para facilitar a construção de um objeto Person.**

* A composição feita através de uma abordagem *hold* (segurar), como no exemplo acima, permite que o usuário use os objetos segurados de forma indireta. Dependendo da aplicação, isto pode ser considerado excessivamente verboso:

In [13]:
thanos.contact_card.mail = "athan@asi.os"

In [14]:
print(thanos)

Name: Thanos, Mail: athan@asi.os, Telefone: 001122334455


* Uma abordagem alternativa de composição é a *wrap* (embrulhar), onde escondemos os objetos embrulhados, expondo de forma direta o que deve ser acessado:

In [15]:
class Person:
    def __init__(self, name, tax_record=None, contact_card=None):
        self.name = name
        self.__tax_record = tax_record
        self.__contact_card = contact_card
    
    def __str__(self):
        base_string = "Name: {}".format(self.name)
        if self.__tax_record:
            base_string += ", {}".format(str(self.__tax_record))
        if self.__contact_card:
            base_string += ", {}".format(str(self.__contact_card))
        return base_string
    
    def hello(self):
        print("Hi, I'm {}".format(self.name))
    
    @property
    def mail(self):
        return self.__contact_card.mail
    
    @mail.setter
    def mail(self, mail):
        self.__contact_card.mail = mail

In [16]:
thanos = Person("Thanos", contact_card=card)

In [17]:
print(thanos)

Name: Thanos, Mail: athan@asi.os, Telefone: 001122334455


In [18]:
thanos.contact_card.mail = "tsou@an.as"

AttributeError: 'Person' object has no attribute 'contact_card'

In [19]:
thanos.mail = "tsou@an.as"

In [20]:
print(thanos)

Name: Thanos, Mail: tsou@an.as, Telefone: 001122334455


## Interfaces versus protocolos

No notebook anterior, vimos diferentes exemplos de classes abstratas e citamos as diferenças entre interfaces, mixins e ABCs.

Esses conceitos são muito usados em linguagens como C++ e Java, que lidam com todos através de diferentes formas de herança múltipla.

Linguagens como Python e Ruby trabalham com polimorfismo de maneira mais informal -- através de **protocolos**.

> In other words, don’t check whether it IS-a duck: check whether it QUACKS-like-a duck, WALKS-like-a duck, etc, etc, depending on exactly what subset of duck-like behaviour you need to play your language-games with.

Fonte: Alex Martelli em uma [discussão em um grupo da Google](https://groups.google.com/forum/?hl=en#!msg/comp.lang.python/CCs2oJdyuzc/NYjla5HKMOIJ). Tradução livre:

> Em outras palavras, não cheque se É um pato: cheque se faz QUACK como um pato, ANDA como um pato, etc, etc, dependendo de com qual subconjunto de comportamentos típicos de patos você quer jogar seus jogos linguísticos. 

O **duck typing** (tipagem pato) descrito por Alex é a base do polimorfismo em Python. 

Existem diferentes formas de trabalharmos com duck typing, então vamos começar revendo os exemplos que usamos no roteiro anterior.

Inicialmente, queremos que nossa classe que representa uma equipe possa informar seu tamanho:

In [21]:
class Team:
    def __init__(self, args=[]):
        self.players = list(args)
    
    def __len__(self):
        return len(self.players)

No exemplo original, dissemos que a classe `Team` herdava da interface `Sized`, que prescrevia o método mágico `__len__`. 

Neste exemplo, apenas implementamos o método mágico diretamente e vemos que o interpretador Python é inteligente o suficiente para entender que um objeto da classe Team é compatível com `Sized`:

In [22]:
equipe1 = Team(["Python", "Ruby"])

In [23]:
from collections.abc import Sized
isinstance(equipe1, Sized)

True

A mesma situação se repete com as interfaces `Container` e `Iterable`:

In [24]:
class Team:
    def __init__(self, args=[]):
        self.players = list(args)
    
    def __len__(self):
        return len(self.players)
    
    def __contains__(self, value):
        return value in self.players

In [25]:
equipe1 = Team(["Python", "Ruby"])

In [26]:
from collections.abc import Container
isinstance(equipe1, Container)

True

In [27]:
class Team:
    def __init__(self, args=[]):
        self.players = list(args)
    
    def __len__(self):
        return len(self.players)
    
    def __contains__(self, value):
        return value in self.players
    
    def __iter__(self):
        return iter(self.players)

In [28]:
equipe1 = Team(["Python", "Ruby"])

In [29]:
from collections.abc import Iterable
isinstance(equipe1, Iterable)

True

O conceito de protocolo é tão importante no Python que o interpretador também identifica que nossa classe implementa a interface `Collection`, que prescreve os métodos `__len__`, `__contains__` e `__iter__`:

**O código abaixo só funciona no Python > 3.5, já que a interface `Collection` foi adicionada no Python 3.6 ;)**

In [30]:
from collections.abc import Collection
isinstance(equipe1, Collection)

ImportError: cannot import name 'Collection'

### Nem tudo é perfeito

Vamos expandir nosso exemplo para o contexto da ABC `Sequence`, que requer a implementação dos métodos mágicos `__getitem__` e `__len__` fornecia os métodos mágicos `__contains__`, `__iter__` e `__reversed__`.

In [31]:
class Team:
    def __init__(self, args=[]):
        self.players = list(args)
    
    def __len__(self):
        return len(self.players)
    
    def __getitem__(self, index):
        return self.players[index]

In [32]:
equipe1 = Team(["Python", "Ruby"])

In [33]:
"Python" in equipe1

True

In [34]:
for lang in equipe1:
    print(lang)

Python
Ruby


In [35]:
len(equipe1)

2

In [36]:
list(reversed(equipe1))

['Ruby', 'Python']

E o que não é perfeito nisso? Bom, veja os resultados das expressões abaixo:

In [37]:
from collections.abc import Sequence
print(isinstance(equipe1, Sequence))
print(isinstance(equipe1, Container))
print(isinstance(equipe1, Iterable))

False
False
False


Como a classe `Team` não implementa diretamente os métodos mágicos `__contains__` e `__iter__`, o interpretador não consegue identificar que ela implementa as interfaces `Container` e `Iterable`.

Pior ainda, mesmo implementando os métodos mágicos `__len__` e `__getitem__`, o interpretador não identifica que a classe `Team` implementa a ABC `Sequence`.

**Ainda estou tentando identificar o motivo da não identificação de `Sequence`. Minha hipótese é que apenas interfaces podem ser identificadas automaticamente -- `Sequence` é uma ABC mas não é uma interface.**

### Lidando bem com protocolos e polimorfismo

O exemplo acima serve como uma boa introdução pra discussão sobre como lidar com polimorfismo em uma linguagem com tipagem dinâmica.

Vamos primeiro listar algumas sugestões que podem ser encontradas na internet:
1. Testar se o tipo do argumento de entrada é o que você espera.
2. Testar se o tipo do argumento pertence a uma determinada família.
3. Tentar usar o argumento diretamente.

Vamos discutir cada um dos pontos, mas antes responda pra você mesmo qual das opções acima você acha mais intuitiva.

#### O tipo que você espera

O exemplo abaixo mostra a situação #1, onde testamos se o tipo do argumento de entrada é o que esperamos:

In [38]:
def custom_range(start, end, step):
    if type(start) == int and type(end) == int and type(step) == int:
        from math import ceil
        for i in range(ceil((end - start) / step)):
            yield start + step * i

In [39]:
list(custom_range(0.5, 3, 0.3))

[]

In [40]:
list(custom_range(0, 3, 1))

[0, 1, 2]

No entanto, quebramos o polimorfismo natural da linguagem! Em alguns casos, isto pode ser necessário, mas nessa situação nosso procedimento gerador poderia aceitar números reais tranquilamente:

In [41]:
def custom_range(start, end, step):
    from math import ceil
    for i in range(ceil((end - start) / step)):
        yield start + step * i

In [42]:
list(custom_range(0.5, 3, 0.3))

[0.5, 0.8, 1.1, 1.4, 1.7, 2.0, 2.3, 2.6, 2.9]

#### Uma família de tipos

É lógico que um exemplo tão simples seria facilmente resolvido acrescentando o tipo `float` aos testes, mas este exemplo é apenas didático 👍🏻

Uma alternativa ao teste direto de tipos é o teste por famílias de tipos. Esse tipo de teste é facilitado pelo polimorfismo por subtipo, que permite que um objeto de uma classe derivada seja tratado como um objeto de sua classe base:

In [43]:
class Person:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return "Name: {}".format(self.name)
    
    def hello(self):
        print("Hi, I'm {}".format(self.name))
        
class TaxPayer(Person):
    def __init__(self, name, cpf, income):
        super().__init__(name)
        self.cpf = cpf
        self.income = float(income)
        
    def __str__(self):
        return "{}, CPF: {}, Renda: {}, Imposto a pagar: {}".format(super().__str__(), self.cpf, self.income, self.duetax)
    
    @property
    def duetax(self):
        return self.income * 0.2

class Contact(Person):
    def __init__(self, name, mail, phone):
        super().__init__(name)
        self.mail = mail
        self.phone = phone
    
    def __str__(self):
        return "{}, Mail: {}, Telefone: {}".format(super().__str__(), self.mail, self.phone)

In [44]:
marcelina = Contact("Marcelina", "marce@li.na", "001122334455")

In [45]:
print(isinstance(marcelina, Contact))
print(isinstance(marcelina, TaxPayer))
print(isinstance(marcelina, Person))

True
False
True


Como era de se esperar, um objeto da classe Contact não é considerado pelo interpretador como relacionado a um objeto da classe TayPayer.

Vamos ver um exemplo de procedimento que só se aplica a pessoas:

In [46]:
def human_rights(obj):
    if isinstance(obj, Person):
        return True
    return False

In [47]:
human_rights(Person("Leonardo"))

True

In [48]:
human_rights(marcelina)

True

In [49]:
juliano = TaxPayer("Juliano", "000.111.222-33", 1500)

In [50]:
human_rights(juliano)

True

In [51]:
human_rights(3)

False

#### A torre numérica do Python

A forma mais extensível de testar se um tipo pertence a determinada família é testando em relação a interfaces (ou ABCs).

O módulo `numbers`, por exemplo, define uma torre numérica para ajudar a identificar se um objeto é um número e quais características ele apresenta:

In [52]:
from numbers import Real
def custom_range(start, end, step):
    if all([isinstance(arg, Real) for arg in (start, end, step)]):
        from math import ceil
        for i in range(ceil((end - start) / step)):
            yield start + step * i

In [53]:
list(custom_range(0.5, 3, 0.3))

[0.5, 0.8, 1.1, 1.4, 1.7, 2.0, 2.3, 2.6, 2.9]

In [54]:
list(custom_range(0, 3, 1))

[0, 1, 2]

No entanto, nem sempre temos à disposição a capacidade de testar nossos objetos contra interfaces e ABCs, como já vimos no caso da classe `Team`:

In [55]:
print(isinstance(equipe1, Sequence))
print(isinstance(equipe1, Container))
print(isinstance(equipe1, Iterable))

False
False
False


De fato, a própria documentação da interface `Iterable` diz que a única forma confiável de testar se um objeto é iterável é tentando iterar sobre ele (o que nos leva à terceira e última estratégia que vamos discutir aqui).

#### Tentando usar o argumento diretamente

Se tentarmos usar instanciar um `range` passando argumentos que não sejam inteiros, recebemos uma mensagem de erro:

In [56]:
range(0.5, 3, 0.3)

TypeError: 'float' object cannot be interpreted as an integer

Isto acontece porque a implementação padrão do Python trata os argumentos de entrada como inteiros, o que leva a exceções quando isso não é verdade.

> It's easier to ask forgiveness than it is to get permission.

Grace Hopper. Tradução livre:

> É mais fácil pedir perdão do que conseguir permissão.

Essa é uma abordagem comum na comunidade Python, tanto que possui um apelido: EAFP (*Easier to Ask Forgiveness than Permission*).

É assim que podemos dizer que nosso procedimento `custom_range` definido sem o auxílio da torre numérica recebe como parâmetro objetos com características de ponto flutuante (*float-like objects*).

Quando não lidamos com operações que só funcionam com tipos específicos (como no caso de `range`), uma alternativa é tentar converter o argumento de entrada em um argumento que apresente a característica requerida pelo seu código.

No caso da interface `Iterable`, por exemplo, podemos tentar convertê-la para uma `list`, que consome de objetos iteráveis e preserva a ordem do conteúdo:
* se a conversão der certo, nosso código funcionará.
* se a conversão der errado, o usuário é informado logo no início da execução do nosso código.

Note que já fazemos isso no construtor da nossa classe `Team`, podemos criar uma nova equipe a partir de outra já existente:

In [57]:
equipe2 = Team(equipe1)

In [58]:
for jogador in equipe2:
    print(jogador)

Python
Ruby


Já se passamos um tipo não iterável, uma excessão será lançada logo de cara:

In [59]:
equipe3 = Team(15)

TypeError: 'int' object is not iterable